In [17]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import json

In [18]:

def hex_to_rgb(s: str) -> tuple:
    s = s.lstrip('#')
    return tuple(int(s[i:i+2], 16) for i in (0, 2, 4))

def plotly_color(s: str, alpha:float = 1)-> str:
    ints = hex_to_rgb(s)
    return f'rgba({ints[0]},{ints[1]},{ints[2]},{alpha})'

# colors_other = ['#17468A', '#4C8DCA', '#78E5EB', '#F5F0F2', '#E12D53']
# color_unum = '#621dd0'
# color_body = plotly_color(color_unum, 0.2) # '#637381'
colors_other = px.colors.sequential.Viridis


In [3]:
def frame_files(file_path: str) -> tuple:
    j = json.loads(open(file_path, 'r').read())
    context = j.get('context', dict())
    benchmarks = j.get('benchmarks', list())

    assert isinstance(context, dict)
    assert isinstance(benchmarks, list)
    for b in benchmarks:
        assert isinstance(b, dict)
        b.update(context)

    return pd.DataFrame(benchmarks)

In [4]:
df = pd.concat([
    frame_files("result_mbp_intel.json"), 
    frame_files("result_mbp_arm.json"), 
    frame_files("result_asus.json"), 
    frame_files("result_server.json"),
], ignore_index=True)

In [5]:
def make_bar(name: str, color: str, ops: list, workloads: list) -> go.Bar:
    x = ops
    text = [name] * len(ops)
    # text =  [f'<b>{machine}: {int(ops[i]):,} bytes/s</b>' for i in range(0, len(ops))]
    return go.Bar(
        name=name, 
        x=workloads, 
        y=x, 
        orientation='v',
        text=text,
        textposition='outside',
        # marker_color=color,
    )

In [6]:

def make_chart_with_bars(bars):
    fig = go.Figure(data=bars)

    # Change the bar mode
    fig.update_layout(
        # font_family='Inter',
        font_color='#637381',
        barmode='group', 
        width=630, # 490 
        height=630, 
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=10,
            pad=4
        ),
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1, # gap between bars of the same location coordinate.
        showlegend=False,
        yaxis_tickangle=-90,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
    )
    # Uncomment to make bars grow from right to left
    # fig.update_xaxes(autorange='reversed')
    fig.update_yaxes(exponentformat='SI')
    return fig



In [7]:
def chart_for_workload(benchmark_name: str, workloads: list):
    bars = [
        make_bar(
            name='💰 50K', 
            color=colors_other[0],
            ops=df[df["name"].str.contains(benchmark_name) & df["host_name"].str.contains("UnumServer")]['bytes/s'].to_list(),
            workloads=workloads,
        ),
        make_bar(
            name=' ARM', 
            color=colors_other[1],
            ops=df[df["name"].str.contains(benchmark_name) & df["host_name"].str.contains("AppleARM")]['bytes/s'].to_list(),
            workloads=workloads,
        ),
        make_bar(
            name=' Intel', 
            color=colors_other[2],
            ops=df[df["name"].str.contains(benchmark_name) & df["host_name"].str.contains("AppleIntel")]['bytes/s'].to_list(),
            workloads=workloads,
        ),
    ]
    return make_chart_with_bars(bars)

In [8]:
fig = chart_for_workload("std::memcpy", workloads=['1 KB', '4 KB', '1 MB', '4 MB'])
fig.write_image("report_memcpy.svg")
fig.show()

In [9]:
fig = chart_for_workload("std::unordered_map.find", workloads=['16 MB', '32 MB', '64 MB', '128 MB', '256 MB'])
fig.write_image("report_std.svg")
fig.show()

In [10]:
fig = chart_for_workload("google::dense_hash_map.find", workloads=['16 MB', '32 MB', '64 MB', '128 MB', '256 MB'])
fig.write_image("report_google.svg")
fig.show()

In [11]:
fig = chart_for_workload("tsl::robin_map.find", workloads=['16 MB', '32 MB', '64 MB', '128 MB', '256 MB'])
fig.write_image("report_tsl.svg")
fig.show()

In [21]:
df = frame_files('result_containers.json')
workloads = ['Construct', 'Insert', 'Find', 'Erase']
bars = [
    make_bar(
        name='std::unordered_map', 
        color=colors_other[0],
        ops=df[df["name"].str.contains('std::unordered_map')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
    make_bar(
        name='tsl::sparse_map', 
        color=colors_other[1],
        ops=df[df["name"].str.contains('tsl::sparse_map')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
    make_bar(
        name='tsl::hopscotch', 
        color=colors_other[2],
        ops=df[df["name"].str.contains('tsl::hopscotch')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
    make_bar(
        name='tsl::robin_map', 
        color=colors_other[3],
        ops=df[df["name"].str.contains('tsl::robin_map')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
    make_bar(
        name='google::dense_hash_map', 
        color=colors_other[4],
        ops=df[df["name"].str.contains('google::dense_hash_map')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
    make_bar(
        name='google::sparse_hash_map', 
        color=colors_other[5],
        ops=df[df["name"].str.contains('google::sparse_hash_map')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
    make_bar(
        name='unum::flat_hash_map', 
        color=colors_other[6],
        ops=df[df["name"].str.contains('unum::flat_hash_map')]['entries/core/s'].to_list(),
        workloads=workloads,
    ),
]
fig = make_chart_with_bars(bars)
fig.write_image("report_containers.svg")
fig.show()